In [1]:
import nibabel as nib
import os
import numpy as np
import pandas as pd

In [51]:
'''
1: Subject ID
2: Rooster ID (4 last digits of subject ID)
3: Age
4: Sexe (0=female, 1=male)
5: Group (CN, EMCI, LMCI, AD)
6: Conversion (-1: CN/AD at baseline, 1: 16 months, 2: 24 months, 3: 36 months, 4: sMCI/no conversion after 36 months, 5: unknown)
7: MMSE
8: RAVLT
9: FAQ
10: CDR-SB
11: ADAS11
12: ADAS13

To separate sMCI and pMCI:
- pMCI: conversion = 1, 2 or 3
- sMCI: conversion = 4
'''

'\n1: Subject ID\n2: Rooster ID (4 last digits of subject ID)\n3: Age\n4: Sexe (0=female, 1=male)\n5: Group (CN, EMCI, LMCI, AD)\n6: Conversion (-1: CN/AD at baseline, 1: 16 months, 2: 24 months, 3: 36 months, 4: sMCI/no conversion after 36 months, 5: unknown)\n7: MMSE\n8: RAVLT\n9: FAQ\n10: CDR-SB\n11: ADAS11\n12: ADAS13\n\nTo separate sMCI and pMCI:\n- pMCI: conversion = 1, 2 or 3\n- sMCI: conversion = 4\n'

In [36]:
header_data = ['Subject ID','Rooster ID', 'Age', 'Sexe', 'Group', 'Conversion', 'MMSE', 'RAVLT', 'FAQ', 'CDR-SB','ADAS11']

In [37]:
data = pd.read_csv('/home/coleger/Desktop/CV/Alz/ADNI1/list_standardized_tongtong_2017.csv')

In [38]:
data.head()

,002_S_0295,295,84.8,1,CN,-1,28,56,0,3.00,3.00.1
0,002_S_0413,413,76.3,0,CN,-1,29,52,0,3.33,3.33
1,002_S_0559,559,79.3,1,CN,-1,30,48,0,6.00,6.00
2,002_S_0619,619,77.5,1,AD,-1,22,19,25,19.33,19.33
3,002_S_0685,685,89.6,0,CN,-1,30,36,0,3.67,3.67
4,002_S_0729,729,65.1,0,MCI,3,27,38,7,6.67,6.67


In [39]:
data.columns = header_data
data.head()

,Subject ID,Rooster ID,Age,Sexe,Group,Conversion,MMSE,RAVLT,FAQ,CDR-SB,ADAS11
0,002_S_0413,413,76.3,0,CN,-1,29,52,0,3.33,3.33
1,002_S_0559,559,79.3,1,CN,-1,30,48,0,6.00,6.00
2,002_S_0619,619,77.5,1,AD,-1,22,19,25,19.33,19.33
3,002_S_0685,685,89.6,0,CN,-1,30,36,0,3.67,3.67
4,002_S_0729,729,65.1,0,MCI,3,27,38,7,6.67,6.67


In [40]:
new_data = data[['Subject ID','Rooster ID', 'Age', 'Sexe', 'Group', 'Conversion']]
new_data.head()

,Subject ID,Rooster ID,Age,Sexe,Group,Conversion
0,002_S_0413,413,76.3,0,CN,-1
1,002_S_0559,559,79.3,1,CN,-1
2,002_S_0619,619,77.5,1,AD,-1
3,002_S_0685,685,89.6,0,CN,-1
4,002_S_0729,729,65.1,0,MCI,3


In [41]:
# Changement des noms pour la colonne conversion
new_data['Conversion'].replace(to_replace =1, value ='pMCI', inplace=True)
new_data['Conversion'].replace(to_replace =2, value ='pMCI', inplace=True)
new_data['Conversion'].replace(to_replace =3, value ='pMCI', inplace=True)

new_data['Conversion'].replace(to_replace =4, value ='sMCI', inplace=True)

# Changement des noms pour la colonne Sexe
new_data['Sexe'].replace(to_replace =0, value ='female', inplace=True)
new_data['Sexe'].replace(to_replace =0, value ='female', inplace=True)


new_data.head()

/tmp/ipykernel_6195/1153290449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Conversion'].replace(to_replace =1, value ='pMCI', inplace=True)
/tmp/ipykernel_6195/1153290449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Conversion'].replace(to_replace =2, value ='pMCI', inplace=True)
/tmp/ipykernel_6195/1153290449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['Conversion'].replace(to_replac

,Subject ID,Rooster ID,Age,Sexe,Group,Conversion
0,002_S_0413,413,76.3,0,CN,-1
1,002_S_0559,559,79.3,1,CN,-1
2,002_S_0619,619,77.5,1,AD,-1
3,002_S_0685,685,89.6,0,CN,-1
4,002_S_0729,729,65.1,0,MCI,pMCI


In [44]:
new_data.Conversion.value_counts()

-1      419
pMCI    171
sMCI    129
5        98
Name: Conversion, dtype: int64

### One hot encoding sur CN et AD sMCI et pMCI

In [49]:
data_clean = pd.get_dummies(data=new_data, columns=['Group','Conversion'])
data_clean.drop('Group_MCI', inplace=True, axis=1)
data_clean.drop('Conversion_-1', inplace=True, axis=1)
data_clean.drop('Conversion_5', inplace=True, axis=1)

data_clean.head()

,Subject ID,Rooster ID,Age,Sexe,Group_AD,Group_CN,Conversion_pMCI,Conversion_sMCI
0,002_S_0413,413,76.3,0,0,1,0,0
1,002_S_0559,559,79.3,1,0,1,0,0
2,002_S_0619,619,77.5,1,1,0,0,0
3,002_S_0685,685,89.6,0,0,1,0,0
4,002_S_0729,729,65.1,0,0,0,1,0


In [50]:
data_clean.rename(columns = {'Group_AD':'AD', 'Group_CN':'CN',
                              'Conversion_pMCI':'pMCI','Conversion_sMCI':'sMCI'}, inplace = True)
data_clean.head()

,Subject ID,Rooster ID,Age,Sexe,AD,CN,pMCI,sMCI
0,002_S_0413,413,76.3,0,0,1,0,0
1,002_S_0559,559,79.3,1,0,1,0,0
2,002_S_0619,619,77.5,1,1,0,0,0
3,002_S_0685,685,89.6,0,0,1,0,0
4,002_S_0729,729,65.1,0,0,0,1,0
